<div dir=rtl>


در این تمرین، یک مدل درخت تصمیم به‌صورت دستی برای داده‌های معروف Iris پیاده‌سازی شد. ابتدا داده‌ها نرمال‌سازی شدند و سپس با استفاده از تابعی بازگشتی، درخت تصمیم با معیار Gini ساخته شد. برای هر گره، بهترین ویژگی و مقدار آستانه به‌صورت بهینه انتخاب شد تا داده‌ها به صورت حداکثری از هم جدا شوند.

بعد از ساخت مدل، آن را روی مجموعه تست ارزیابی کردیم و معیارهای مختلفی از جمله دقت، precision، recall و f1-score محاسبه شدند. تمام این مقادیر بالا بودند که نشان می‌دهد درخت تصمیم توانسته به‌خوبی ساختار داده را یاد بگیرد و تعمیم بدهد.

ساختار نهایی درخت نیز ساده ولی مؤثر بود؛ به‌گونه‌ای که کلاس Setosa تنها با یک شرط از بقیه جدا شد و برای تفکیک Versicolor و Virginica از دو ویژگی دیگر با چند تقسیم اضافی استفاده شد. این نتایج نشان می‌دهد که درخت تصمیم با عمق محدود هم توانسته ساختار داده‌ها را به‌خوبی یاد بگیرد و دسته‌بندی دقیقی انجام دهد.


</div>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
# from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score


<div dir=rtl>
ما توی این تمرین باید یه SVM (ماشین بردار پشتیبان) رو از صفر پیاده‌سازی می‌کردیم تا دیتاست Iris رو دسته‌بندی کنیم. اول از همه اومدیم مسئله رو ساده کردیم و تبدیلش کردیم به یه مسئله دوتایی (باینری). یعنی فقط گونه‌ی Setosa رو با مقدار ۱ در نظر گرفتیم و بقیه‌ی گونه‌ها رو با ‎‑۱، چون توی حالت چندکلاسه کدنویسی سخت‌تر می‌شد. بعد رفتیم سراغ اینکه مدل SVM رو با استفاده از تابع hinge loss و گرادیان نزولی (gradient descent) آموزش بدیم.

</div>

In [8]:

data = datasets.load_iris()
X0 = data.data
y0 = data.target
y0 = np.where(y0 == 0, 1, -1)

Xtr, Xte, ytr, yte = train_test_split(X0, y0, test_size=0.2, random_state=42)


<div dir=rtl>
این تابع برای افزایش ویژگی‌ها با کرنل چندجمله‌ای (Polynomial) هست. اگه poly رو True بزاریم، داده‌ها به درجه‌های بالاتر مثل x² و x³ هم گسترش پیدا می‌کنن تا مدل بتونه مرزهای غیرخطی یاد بگیره. ولی ما فعلاً خاموش گذاشتیم (poly_on = False) چون SVM با داده Setosa به‌خوبی خطی جدا می‌شه.
</div>

In [9]:
def feat(x, poly=False, deg=2):
    if not poly:
        return x
    z = x.copy()
    for d in range(2, deg + 1):
        z = np.hstack((z, x ** d))
    return z


<div dir=rtl>
این تابع hinge loss رو حساب می‌کنه که نشون می‌ده چقدر مدل از نظر فاصله گرفتن از نقاط اشتباه کار می‌کنه. اگر y * (w·x + b) بزرگتر مساوی ۱ باشه یعنی نمونه به‌خوبی جدا شده، ولی اگر کوچکتر باشه جریمه داره.
</div>

In [10]:
def hinge(X, y, w, b, lam):
    m = X.shape[0]
    margins = 1 - y * (X.dot(w) + b)
    loss = np.maximum(0, margins).mean() + lam * np.dot(w, w)
    return loss

<div dir=rtl>
برای هر epoch (دور) و هر نمونه از داده، چک کردیم که شرط margin رعایت شده یا نه. اگه رعایت نشده بود وزن و بایاس رو با فرمول مخصوص به‌روزرسانی کردیم.
</div>

In [11]:

data = datasets.load_iris()
X0 = data.data
y0 = data.target
y0 = np.where(y0 == 0, 1, -1)

Xtr, Xte, ytr, yte = train_test_split(X0, y0, test_size=0.2, random_state=42)



poly_on = False
Xtr = feat(Xtr, poly_on, 2)
Xte = feat(Xte, poly_on, 2)

w = np.zeros(Xtr.shape[1])
b = 0
lr = 1e-3
lam = 1e-2
epoch = 1000



# lr = "0.01"
# w = np.ones(99)
# for i in range(epoch): pass

for _ in range(epoch):
    for i, xi in enumerate(Xtr):
        cond = ytr[i] * (np.dot(xi, w) + b)
        if cond >= 1:
            dw = 2 * lam * w
            db = 0
        else:
            dw = 2 * lam * w - ytr[i] * xi
            db = -ytr[i]
        w -= lr * dw
        b -= lr * db

loss_val = hinge(Xte, yte, w, b, lam)

def pred(X):
    return np.sign(X.dot(w) + b)

yp = pred(Xte)
acc = (yp == yte).mean()
cm = confusion_matrix(yte, yp, labels=[1, -1])
pr = precision_score(yte, yp, pos_label=1, zero_division=0)
rc = recall_score(yte, yp, pos_label=1, zero_division=0)
f1 = f1_score(yte, yp, pos_label=1, zero_division=0)

print("acc:", acc)
print("loss:", loss_val)
print("cm:\n", cm)
print("prec:", pr)
print("rec:", rc)
print("f1:", f1)


acc: 1.0
loss: 0.01163211724797394
cm:
 [[10  0]
 [ 0 20]]
prec: 1.0
rec: 1.0
f1: 1.0
